### 创建新会话
screen -S training

### 在screen中运行训练命令
python scripts/train_mistral.py --config="/root/autodl-tmp/zishu-sensei/config/training_config.json" --model_config="/root/autodl-tmp/zishu-sensei/config/model_config.json"

### 分离会话(保持运行): Ctrl+A 然后 D
重新连接会话: 
screen -r training

### 如果忘记会话名:
screen -ls  # 查看所有会话

您的screen会话"training_mistral"已存在但处于"Attached"状态，意味着它已经连接到另一个终端。

解决方法:

1. 强制重新连接:
   ```bash
   screen -d -r training_mistral
   ```
   这会先分离(-d)已连接的会话，然后重新连接(-r)

2. 如果上面不起作用，可以用会话ID连接:
   ```bash
   screen -d -r 2630
   ```

这种情况通常是由于之前的SSH连接异常中断，没有正常分离screen会话造成的。使用-d参数强制分离并重新连接即可。

要查看当前所在的screen会话信息，可以使用：

```bash
echo $STY
```

这会显示当前screen会话的ID和名称。例如`2630.training_mistral`。

另外，也可以检查：

```bash
screen -ls | grep \*
```

在某些版本的screen中，当前连接的会话会用星号(*)标记。

如果您确定在screen中但不知道是哪个，可以按 `Ctrl+A` 然后按 `i`，这会在屏幕底部显示当前会话信息。

每次运行时使用不同的输出目录：
python scripts/prepare_data.py --output_dir ./data/train_v2

cd /root/autodl-tmp/zishu-sensei && python scripts/dialogue_gen.py --chars_per_type 500 --concurrent_limit 3 --create_training_data --complexity_simple 0.2 --complexity_medium 0.5 --complexity_complex 0.2 --complexity_advanced 0.1

cd /root/autodl-tmp/zishu-sensei && screen -S zishu_20k_training -X stuff 'python scripts/train_mistral.py --max_samples 20000\n'

## 训练计划

#### 需要完成GPU并行训练
#### 需要完成统一兼容的训练脚本
#### 自动化训练：爬虫入口->数据获取->数据清洗->检测数据质量->进行并行训练(6张V100显卡)
#### 建立关于二次元能力的评估模型体系
#### 训练完之后，自动评估模型，评估后如果低于期望值，那么自动停止
#### 有训练中断恢复机制，如果训练意外中断，那么自动回到最近检查点继续开始，如果尝试多次不行，那么训练停止


## Zishu-sensei AI训练系统详细实施计划 (6×V100版本)

## 阶段1：基础架构搭建 (第1-2周)

### 1.1 核心训练框架文件
**目录**: `src/training/`
- `distributed_trainer.py` - 6GPU分布式训练主控制器
- `gpu_manager.py` - GPU资源分配和监控管理
- `model_parallel.py` - 模型并行策略实现
- `data_parallel.py` - 数据并行策略实现
- `mixed_parallel.py` - 混合并行策略(2x3或3x2配置)
- `memory_optimizer.py` - 显存优化和管理
- `communication_backend.py` - GPU间通信优化

### 1.2 配置管理文件
**目录**: `config/training/`
- `distributed_config.json` - 6GPU分布式训练配置
- `model_scaling_config.json` - 不同模型规模配置(7B/13B/30B)
- `memory_config.json` - 显存分配和优化配置
- `performance_config.json` - 性能调优参数
- `resource_allocation.json` - GPU资源分配策略

### 1.3 监控和日志文件
**目录**: `src/monitoring/`
- `gpu_monitor.py` - 6GPU实时监控
- `performance_tracker.py` - 训练性能跟踪
- `resource_logger.py` - 资源使用日志
- `alert_system.py` - 异常告警系统

## 阶段2：数据流水线系统 (第3-5周)

### 2.1 数据获取模块
**目录**: `src/data_pipeline/acquisition/`
- `bangumi_api_client.py` - Bangumi官方API客户端
- `public_dataset_loader.py` - 公开数据集加载器
- `community_data_collector.py` - 社区数据收集器
- `data_source_manager.py` - 多数据源统一管理
- `rate_limiter.py` - API请求限流控制
- `compliance_checker.py` - 数据获取合规检查

### 2.2 数据清洗和预处理
**目录**: `src/data_pipeline/processing/`
- `text_normalizer.py` - 文本标准化处理
- `deduplicator.py` - 重复数据检测和去除
- `quality_filter.py` - 数据质量过滤
- `content_classifier.py` - 内容分类和标记
- `toxicity_detector.py` - 有害内容检测
- `format_converter.py` - 多格式数据转换
- `batch_processor.py` - 大批量数据处理器

### 2.3 数据质量评估
**目录**: `src/data_pipeline/quality/`
- `relevance_scorer.py` - 二次元相关性评分
- `diversity_analyzer.py` - 数据多样性分析
- `completeness_checker.py` - 数据完整性检查
- `bias_detector.py` - 偏见内容检测
- `statistical_analyzer.py` - 数据统计分析
- `quality_report_generator.py` - 质量报告生成器

## 阶段3：高效分布式训练系统 (第6-9周)

### 3.1 训练策略实现
**目录**: `src/training/strategies/`
- `strategy_selector.py` - 自动选择最优并行策略
- `four_gpu_strategy.py` - 4GPU训练策略(主要训练)
- `six_gpu_strategy.py` - 6GPU全力训练策略
- `hybrid_strategy.py` - 4+2混合策略(训练+评估)
- `memory_efficient_strategy.py` - 显存优化策略
- `communication_optimized.py` - 通信优化策略

### 3.2 模型规模适配
**目录**: `src/models/`
- `model_factory.py` - 模型工厂类
- `mistral_7b_config.py` - Mistral 7B配置
- `llama_13b_config.py` - LLaMA 13B配置  
- `large_model_30b_config.py` - 30B大模型配置
- `custom_architecture.py` - 自定义架构支持
- `model_sharding.py` - 模型分片管理

### 3.3 检查点和恢复系统
**目录**: `src/training/checkpoint/`
- `checkpoint_manager.py` - 检查点管理器
- `state_saver.py` - 训练状态保存
- `recovery_engine.py` - 自动恢复引擎
- `backup_scheduler.py` - 备份调度器
- `corruption_detector.py` - 损坏检测器
- `rollback_manager.py` - 回滚管理器

### 3.4 训练中断和重启
**目录**: `src/training/resilience/`
- `failure_detector.py` - 故障检测器
- `auto_restart.py` - 自动重启系统
- `health_monitor.py` - 健康状态监控
- `graceful_shutdown.py` - 优雅关闭处理
- `retry_controller.py` - 重试控制器
- `emergency_save.py` - 紧急保存机制

## 阶段4：二次元专业评估体系 (第10-12周)

### 4.1 评估数据集构建
**目录**: `evaluation/datasets/`
- `anime_knowledge_test.json` - 动漫知识测试集
- `character_recognition_test.json` - 角色识别测试
- `plot_understanding_test.json` - 剧情理解测试
- `cultural_context_test.json` - 文化背景测试
- `terminology_test.json` - 专业术语测试
- `roleplay_scenarios.json` - 角色扮演场景

### 4.2 评估指标系统
**目录**: `src/evaluation/metrics/`
- `otaku_score_calculator.py` - 二次元能力综合评分
- `knowledge_accuracy.py` - 知识准确性评估
- `cultural_sensitivity.py` - 文化敏感性评估
- `personality_consistency.py` - 人格一致性评估
- `interaction_quality.py` - 交互质量评估
- `creativity_evaluator.py` - 创造性评估

### 4.3 自动化评估系统
**目录**: `src/evaluation/automation/`
- `evaluation_scheduler.py` - 评估调度器
- `benchmark_runner.py` - 基准测试执行器
- `result_analyzer.py` - 结果分析器
- `trend_tracker.py` - 性能趋势跟踪
- `threshold_monitor.py` - 阈值监控器
- `report_generator.py` - 评估报告生成

### 4.4 人工评估接口
**目录**: `src/evaluation/human/`
- `expert_interface.py` - 专家评估界面
- `crowd_evaluation.py` - 众包评估系统
- `feedback_collector.py` - 反馈收集器
- `annotation_validator.py` - 标注验证器

## 阶段5：智能训练控制系统 (第13-14周)

### 5.1 性能监控和预警
**目录**: `src/control/monitoring/`
- `real_time_monitor.py` - 实时性能监控
- `anomaly_detector.py` - 异常检测器
- `performance_predictor.py` - 性能预测器
- `bottleneck_analyzer.py` - 瓶颈分析器
- `alert_dispatcher.py` - 告警分发器

### 5.2 自动决策系统
**目录**: `src/control/decision/`
- `training_controller.py` - 训练控制器
- `stop_condition_checker.py` - 停止条件检查器
- `strategy_switcher.py` - 策略切换器
- `resource_reallocator.py` - 资源重新分配器
- `emergency_handler.py` - 紧急情况处理器

### 5.3 资源管理和优化
**目录**: `src/control/resources/`
- `gpu_scheduler.py` - GPU调度器
- `memory_manager.py` - 内存管理器
- `cost_optimizer.py` - 成本优化器
- `workload_balancer.py` - 工作负载均衡器
- `power_manager.py` - 功耗管理器

## 阶段6：端到端集成和测试 (第15-16周)

### 6.1 系统集成
**目录**: `src/integration/`
- `pipeline_orchestrator.py` - 流水线编排器
- `service_coordinator.py` - 服务协调器
- `dependency_manager.py` - 依赖管理器
- `configuration_validator.py` - 配置验证器
- `system_initializer.py` - 系统初始化器

### 6.2 测试套件
**目录**: `tests/`
- `unit/test_distributed_training.py` - 分布式训练单元测试
- `unit/test_data_pipeline.py` - 数据流水线测试
- `unit/test_evaluation_system.py` - 评估系统测试
- `integration/test_6gpu_training.py` - 6GPU集成测试
- `integration/test_end_to_end.py` - 端到端测试
- `performance/test_scalability.py` - 可扩展性测试
- `stress/test_long_running.py` - 长时间运行测试

### 6.3 部署和运维
**目录**: `deployment/`
- `docker/Dockerfile.training` - 训练环境Docker文件
- `scripts/setup_environment.sh` - 环境设置脚本
- `scripts/start_training.sh` - 训练启动脚本
- `monitoring/prometheus_config.yml` - 监控配置
- `monitoring/grafana_dashboard.json` - 监控面板
- `backup/backup_strategy.yml` - 备份策略配置

### 6.4 文档和使用指南
**目录**: `docs/`
- `installation_guide.md` - 安装指南
- `6gpu_training_guide.md` - 6GPU训练指南
- `configuration_reference.md` - 配置参考
- `troubleshooting.md` - 故障排除
- `performance_tuning.md` - 性能调优
- `api_documentation.md` - API文档

## 关键里程碑和交付物

| 周次 | 阶段 | 主要交付物 |
|------|------|------------|
| 1-2 | 基础架构 | 6GPU分布式训练框架 |
| 3-5 | 数据流水线 | 自动化数据处理系统 |
| 6-9 | 分布式训练 | 高效训练和恢复系统 |
| 10-12 | 评估体系 | 二次元专业评估系统 |
| 13-14 | 智能控制 | 自动化训练控制系统 |
| 15-16 | 集成测试 | 完整可部署系统 |

## 资源分配策略
- **主要训练**: 4-6 GPU (根据模型规模调整)
- **数据处理**: 1-2 GPU 
- **评估测试**: 1-2 GPU
- **监控备用**: 灵活分配剩余资源

这个计划充分利用了6张V100的优势，确保了系统的高效性、稳定性和可扩展性。

# 🎭 二次元对话数据生成指南

## 📋 概述

本文档介绍如何使用AI自动生成高质量的二次元对话训练数据，支持多种方式和API提供商。

## 🚀 快速开始

### 方法1: 简化版生成（推荐新手）

```bash
# 快速生成少量测试数据
python scripts/simple_dialogue_gen.py --num_per_character 5 --create_training_data

# 如果有千问API密钥，质量更好
python scripts/simple_dialogue_gen.py --num_per_character 10 --api_key your-qwen-key --create_training_data
```

### 方法2: 完整版生成（推荐批量）

```bash
# 大规模生成（需要配置API）
python scripts/generate_dialogue_data.py --num_per_character 50 --create_dataset
```

## 🔧 配置说明

### API配置文件 (`config/api_config.json`)

```json
{
  "providers": {
    "qwen": {
      "api_key": "your-actual-qwen-key",
      "enabled": true,
      "description": "通义千问，中文对话质量优秀"
    },
    "deepseek": {
      "api_key": "your-deepseek-key", 
      "enabled": false
    },
    "local": {
      "base_url": "http://localhost:8000/v1",
      "enabled": true,
      "description": "本地模型，免费但需自己部署"
    }
  }
}
```

### 支持的API提供商

| 提供商 | 中文能力 | 成本 | 推荐度 |
|--------|----------|------|--------|
| **通义千问** | ⭐⭐⭐⭐⭐ | ¥0.001/1K tokens | 🥇 强烈推荐 |
| **DeepSeek** | ⭐⭐⭐⭐ | ¥0.0014/1K tokens | 🥈 性价比高 |
| **智谱GLM** | ⭐⭐⭐⭐ | ¥0.001/1K tokens | 🥉 质量稳定 |
| **本地模型** | ⭐⭐⭐ | 免费 | 💡 节约成本 |

## 📊 生成的角色类型

脚本支持以下二次元经典角色类型：

- **傲娇**: 表面高傲内心温柔，经常说反话
- **元气少女**: 活泼开朗，充满活力  
- **中二病**: 自认特殊能力，用词夸张
- **大小姐**: 优雅贵族气质，偶尔天然
- **冷酷**: 性格冷淡，不善表达
- **学霸**: 聪明好学，偶尔呆萌

## 🎬 场景覆盖

- **校园日常**: 教室、图书馆、食堂、社团活动
- **日常生活**: 购物、咖啡厅、看电影、散步
- **兴趣爱好**: 讨论动漫、游戏、读书、创作
- **情感互动**: 安慰、感谢、道歉、庆祝
- **特殊场景**: 文化祭、暑假、雨天、意外相遇

## 💰 成本估算

### 千问API成本 (推荐)
```
生成500个对话 ≈ 50万tokens ≈ ¥0.5
生成2000个对话 ≈ 200万tokens ≈ ¥2
```

### 免费方案
```
1. 使用本地部署的模型 (需要显卡)
2. 使用免费API额度
3. 混合使用：千问生成高质量样本 + 本地生成批量数据
```

## 📁 输出格式

### 原始对话数据
```json
{
  "character_type": "傲娇",
  "scenario": "在教室讨论作业", 
  "turns": [
    {
      "user": "这道数学题好难啊，你会做吗？",
      "character": "哼！这种简单的题目你都不会？真是笨蛋呢...不过，既然你这么诚恳地请教，本小姐就勉为其难地教你一下吧！"
    }
  ]
}
```

### 训练数据格式
```json
{
  "instruction": "你是一个傲娇类型的二次元角色，当前场景是：在教室讨论作业。用户对你说：这道数学题好难啊，你会做吗？",
  "input": "",
  "output": "哼！这种简单的题目你都不会？真是笨蛋呢...不过，既然你这么诚恳地请教，本小姐就勉为其难地教你一下吧！",
  "character_type": "傲娇",
  "scenario": "在教室讨论作业"
}
```

## 🎯 质量控制

脚本内置多重质量保证：

1. **角色特征检测**: 确保生成的对话符合角色设定
2. **对话长度控制**: 过短或过长的回复会被过滤
3. **内容去重**: 避免重复的表达方式
4. **场景一致性**: 确保对话内容符合设定场景
5. **多轮连贯性**: 保证对话的逻辑连续性

## 🔄 使用流程

### Step 1: 准备环境
```bash
pip install aiohttp requests
mkdir -p data/generated_dialogues
```

### Step 2: 配置API
```bash
# 编辑配置文件，添加你的API密钥
vim config/api_config.json
```

### Step 3: 快速测试
```bash
# 生成少量数据测试
python scripts/simple_dialogue_gen.py --num_per_character 3
```

### Step 4: 批量生成
```bash
# 大规模生成
python scripts/generate_dialogue_data.py --num_per_character 100
```

### Step 5: 集成训练
```bash
# 将生成的数据添加到训练脚本
python scripts/train_mistral.py --additional_data data/generated_dialogues/training_data_*.jsonl
```

## 🎨 自定义扩展

### 添加新角色类型
```python
# 在脚本中添加新角色
self.characters["新角色"] = {
    "personality": "角色性格描述",
    "speech_patterns": ["特征词1", "特征词2"],
    "topics": ["话题1", "话题2"]
}
```

### 添加新场景
```python
# 添加新的对话场景
self.scenarios["新分类"] = [
    "具体场景1",
    "具体场景2"
]
```

## 📈 效果优化建议

1. **混合数据源**: 结合AI生成 + 人工精选的高质量样本
2. **渐进式训练**: 先用少量数据验证效果，再大规模生成
3. **质量过滤**: 人工审查关键样本，确保质量标准
4. **角色平衡**: 确保各角色类型的数据量均衡
5. **场景多样**: 覆盖尽可能多的对话场景

## ⚠️ 注意事项

- API调用可能有频率限制，注意控制并发数
- 生成的内容需要人工抽查质量
- 建议先小规模测试，确认效果后再大规模生成
- 保存好API密钥，不要泄露到代码仓库
- 定期备份生成的数据，避免意外丢失

## 🔗 相关资源

- [训练脚本文档](docs/training.md)
- [评估工具文档](docs/evaluation.md) 
- [API提供商申请指南](docs/api_providers.md)

---

💡 **小贴士**: 推荐先用千问生成少量高质量种子数据，然后用这些数据训练一个小模型，再用小模型生成更多数据，实现数据的"自举"式增长。 

cd autodl-tmp/zishu-sensei/scripts && python evaluate.py --eval_type all --baseline_path ../output_8k_baseline/final_model

## 桌面宠物功能开发步骤计划

### 阶段1：基础框架 (1-2周)

1. **创建基础目录结构**
   - 确认 `src/interface/desktop_pet/` 目录存在
   - 确认 `src/character/appearance/` 目录存在

2. **创建基础文件**
   - `src/interface/desktop_pet/__init__.py`
   - `src/interface/desktop_pet/pet.py` - 桌面宠物主类
   - `src/interface/desktop_pet/renderer.py` - 渲染引擎

3. **配置文件完善**
   - 编辑 `config/desktop_config.json` - 添加窗口设置、动画帧率等参数
   - 编辑 `config/character/default.json` - 角色基本设置

### 阶段2：角色外观系统 (2-3周)

1. **创建外观相关文件**
   - `src/character/appearance/base.py` - 基础外观类
   - `src/character/appearance/accessories.py` - 头饰系统
   - `src/character/appearance/styles.py` - 外观风格管理

2. **创建情感系统文件**
   - `src/character/emotions.py` - 情感状态管理

3. **准备资源文件**
   - 在 `assets/images/character/` 中添加角色基础图像
   - 在 `assets/images/accessories/` 中添加头饰图像
   - 在 `assets/animations/` 中添加动画序列

### 阶段3：交互系统 (2周)

1. **创建交互系统文件**
   - `src/interface/desktop_pet/interaction.py` - 事件响应系统
   - `src/interface/desktop_pet/tray.py` - 系统托盘集成

2. **菜单和控制文件**
   - `src/interface/desktop_pet/menu.py` - 右键菜单系统

3. **配置文件更新**
   - 更新 `config/desktop_config.json` - 添加交互相关设置

### 阶段4：行为模式 (2周)

1. **创建行为系统文件**
   - `src/interface/desktop_pet/behavior.py` - 行为状态机
   - `src/interface/desktop_pet/states.py` - 各种状态实现

2. **动画控制文件**
   - `src/interface/desktop_pet/animation.py` - 动画控制系统

3. **配置文件更新**
   - 在 `config/character/default.json` 中添加行为相关配置
   - 在 `config/desktop_config.json` 中添加状态切换配置

### 阶段5：AI集成 (1-2周)

1. **创建UI组件文件**
   - `src/interface/desktop_pet/chat_ui.py` - 对话气泡UI

2. **创建AI接口文件**
   - `src/interface/desktop_pet/ai_proxy.py` - AI响应代理

3. **配置文件更新**
   - 在 `config/desktop_config.json` 中添加AI集成配置
   - 在 `config/model_config.json` 中添加桌面模式推理配置

### 阶段6：集成与测试 (1周)

1. **创建启动文件**
   - 更新 `src/interface/__init__.py` - 添加桌面宠物初始化代码
   - 更新 `app.py` - 确保桌面模式正确调用

2. **编写测试文件**
   - `tests/unit/test_desktop.py` - 桌面宠物单元测试
   - `tests/integration/test_desktop_integration.py` - 集成测试

3. **完成文档**
   - 编写 `docs/desktop_pet.md` - 桌面宠物功能文档

### 准备工作

1. **资源收集**
   - 收集或创建角色基础图像
   - 准备各种头饰图像（蝴蝶结、小乌龟、花朵等）
   - 准备多种表情和动画序列

2. **环境配置**
   - 确保已安装PyQt6或PySide6
   - 安装Pillow图像处理库

3. **示例对话数据**
   - 准备预设对话样本，在AI模型集成前使用

这个计划可以在模型训练的同时并行执行，专注于创建必要的文件结构和基础功能，等待模型训练完成后再进行AI集成。

## 核心服务和事件总线待完成。

cd /root/autodl-tmp/zishu-sensei && git remote set-url origin git@github.com:erliu8/Zishu-sensei-new.git && echo "Host github.com\n  IdentityFile ~/.ssh/github_deploy_key\n  StrictHostKeyChecking no" > ~/.ssh/config && chmod 600 ~/.ssh/config


cd /root/autodl-tmp/zishu-sensei && git push -u origin master

Bangumi 番组计划：
这是最著名和最全面的中文动漫、游戏、音乐、书籍数据库之一。
它有公开的API，允许开发者获取其海量的结构化数据。
很多开源项目都基于Bangumi的数据。您可以在GitHub上搜索 Bangumi 或 番组计划 找到相关的数据集或API封装库。
萌娘百科 (Moegirlpedia)：
一个内容极其丰富的中文ACGN（动画、漫画、游戏、小说）维基百科。
可以通过维基百科的数据导出工具（dump）获取全站数据，然后进行解析和清洗。数据量非常大，但需要自己处理。
内容主要是非结构化的文本，适合用于训练语言模型。
Bilibili (B站)：
作为中国最大的动漫视频网站，B站有大量的动漫信息、评论和用户数据。
虽然没有官方的公开数据集，但有很多第三方的爬虫项目可以获取动漫的元数据、评论、弹幕等。可以在GitHub搜索 bilibili spider 或 bilibili-api。
豆瓣 (Douban)：
豆瓣电影/读书/音乐板块包含了很多动漫条目和用户评分、评论。
同样可以通过爬虫或非官方API获取数据。

cd /root/autodl-fs && wget -O bangumi-latest.zip "https://github.com/bangumi/Archive/releases/download/archive/dump-2025-04-01.210252Z.zip"

cd /root/autodl-fs && aria2c -x 16 -s 16 -k 1M -o bangumi-latest.zip "https://github.com/bangumi/Archive/releases/download/archive/dump-2025-04-01.210252Z.zip"